In [1]:
import sys
sys.path.append("../")

In [2]:
from docmesh.embedding import EmbeddingModelFactory
from docmesh.vector_store import VectorStoreFactory

from docmesh.llm import LLMFactory
from docmesh.config import Config
from docmesh.qa_bot import QAService

In [3]:
# EmbeddingModelFactory를 통해 임베딩 모델 생성
embedding_model = EmbeddingModelFactory.create_embedding_model(
    provider="openai", model_name="text-embedding-ada-002"
)
# VectorStoreFactory를 통해 벡터 스토어 생성
vector_store = VectorStoreFactory.create_vector_store(
    provider="faiss", embedding_model=embedding_model, path="langchain_doc.faiss", 
)

vector_store.load()

# LLM 서비스 생성 (팩토리 사용)
llm = LLMFactory.create_llm_service(
    provider=Config.LLM_PROVIDER,
    model=Config.LLM_MODEL,
    temperature=Config.LLM_TEMPERATURE,
)

retriever = vector_store.vectorstore.as_retriever()

bot = QAService(llm, retriever)
bot.answer_question("langchain은 무엇입니까?")

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


'langchain은 LangChain 프레임워크의 일부로, LLM(언어 모델) 애플리케이션 및 지능적 에이전트를 추적하고 평가하여 프로토타입에서 제품으로 이동하는 데 도움을 주는 LangSmith와 함께 사용되는 상태 유지형, 다중 액터 애플리케이션을 구축하는 LangGraph입니다. LangGraph은 LangChain과 원활하게 통합되지만 독립적으로 사용할 수 있으며, 많은 기업들이 신뢰하는 생산용 에이전트를 제공합니다.\n\nSources: https://python.langchain.com/docs/integrations/providers/'

In [53]:
import pandas as pd
from typing import List

df = pd.read_csv("ragas_synthetic_dataset.csv")
df['reference_contexts'] = df['reference_contexts'].map(eval)

In [56]:
from datasets import Dataset

test_dataset = Dataset.from_pandas(df)
test_dataset = test_dataset.add_column("retrieved_contexts", list(test_dataset['reference_contexts']))
test_dataset

Dataset({
    features: ['user_input', 'reference_contexts', 'reference', 'synthesizer_name', 'retrieved_contexts'],
    num_rows: 10
})

In [57]:
list_anwser = list()
for q in test_dataset["user_input"]:
    a = bot.answer_question(q)
    list_anwser.append(a)

In [58]:
test_dataset = test_dataset.add_column("answer", list_anwser)

In [45]:
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

In [59]:
result = evaluate(
    dataset=test_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

In [60]:
result

{'context_precision': 0.9000, 'faithfulness': 0.3136, 'answer_relevancy': 0.5722, 'context_recall': 0.9000}